In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep


In [4]:
# 空のDataFrameを作成（追加項目を含む）
columns = ["物件名", "住所", "アクセス", "築年数", "階数", "家賃", "管理費", "間取り", "広さ", "敷金", "礼金", "物件画像URL", "物件詳細URL"]
df = pd.DataFrame(columns=columns)

# ベースとなるURL（ページ番号を含む）
base_url = "https://bridal.suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50"

# 1ページから235ページまでスクレイピング
for page_num in range(1, 236):
    # ページ番号をURLに組み込む
    url = f"{base_url}&page={page_num}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    sleep(1)

    # 各物件の情報をループ
    for property_row in soup.find_all('div', class_='cassetteitem'):
        property_name = property_row.find('div', class_='cassetteitem_content-title').text.strip()  # 物件名
        address = property_row.find('li', class_='cassetteitem_detail-col1').text.strip()  # 住所
        access = ' '.join([div.text.strip() for div in property_row.find_all('div', class_='cassetteitem_detail-text')])  # アクセス
        age = property_row.find('li', class_='cassetteitem_detail-col3').div.text.strip()  # 築年数

        # 物件画像URLを取得（imgタグが存在しない場合はデフォルト値を設定）
        img_tag = property_row.find('img', class_='js-linkImage')
        image_url = img_tag['rel'] if img_tag else "画像なし"

        # 同じ物件の異なる部屋情報を取得
        for room_row in property_row.find_all('tbody'):
            for tr in room_row.find_all('tr'):
                floor = tr.find_all('td')[2].text.strip()  # 階数
                rent = tr.find_all('td')[3].text.strip()  # 家賃
                management_fee = tr.find_all('td')[4].text.strip() if tr.find_all('td')[4].text.strip() != '-' else '0円'  # 管理費
                deposit_and_etc = tr.find_all('td')[5].text.strip().split('/')  # 敷/礼/保証/敷引,償却
                deposit = deposit_and_etc[0] if len(deposit_and_etc) > 0 else '0円'  # 敷金
                reikin = deposit_and_etc[1] if len(deposit_and_etc) > 1 else '0円'  # 礼金
                layout = tr.find_all('td')[6].text.strip()  # 間取り
                size = tr.find_all('td')[7].text.strip()  # 広さ
                detail_url = 'https://bridal.suumo.jp' + tr.find('a', target='_blank')['href']  # 物件詳細URL

                # 新しい行を作成してDataFrameに追加
                new_row = pd.DataFrame([[property_name, address, access, age, floor, rent, management_fee, layout, size, deposit, reikin, image_url, detail_url]], columns=columns)
                df = pd.concat([df, new_row], ignore_index=True)

# DataFrameをCSVファイルに出力
csv_file = 'suumo.csv' 
df.to_csv(csv_file, index=False, encoding='utf-8-sig')
